In [1]:
import json
import time
import datetime
import requests 
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
import pickle 

# API Key IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW
# 1000 Calls / Hour
# Put API Key in file. Read in API key from file instead of hardcoding in jupyter notebook. 
# http://petstore.swagger.io/
# https://api.open.fec.gov/swagger/

In [2]:
# CONFIG_FILE = Path('.')/Path('0-setup')/Path('fec_config.yaml')
# with open(CONFIG_FILE, 'r') as f:
#     config = yaml.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '0-setup\\fec_config.yaml'

In [3]:
# api_key = config['fec']['api_key']

# Interfacing with API 
Goal: Input: Candidate, Year, District:


Output: Overall Financials, Financials in Given District 


Desired Format: .csv


May have to use: node.js, puppeter, AWS, Python Threading (? Later on ?)

### Construct Candidate_id DataFrame A

In [411]:
#Retrieve Comprehensive Candidate Dictionary

i = 1
candidate_dict = []
while True:
    candidate_id = requests.get('https://api.open.fec.gov/v1/candidates/?sort=name\
&has_raised_funds=true\
&min_first_file_date=1980\
&per_page=100\
&api_key=' + api_key + '\
&page='+str(i))
    if candidate_id.json()['results'] ==  []:
        break
    else: 
        results = candidate_id.json()['results']
        candidate_dict = candidate_dict +  results
        i += 1
print('Done')
print(i)

Done
188


In [412]:
df_general_cand = pd.DataFrame(candidate_dict)
df_general_cand.head()

,active_through,candidate_id,candidate_status,cycles,district,district_number,election_districts,election_years,federal_funds_flag,first_file_date,...,incumbent_challenge_full,last_f2_date,last_file_date,load_date,name,office,office_full,party,party_full,state
0,2018,S2UT00229,C,"[2012, 2014, 2016, 2018]",00,0.0,"[00, 00]","[2012, 2018]",False,2012-02-08,...,Open seat,2018-04-23,2018-04-23,2018-08-01T00:13:22,"AALDERS, TIMOTHY NOEL",S,Senate,CON,CONSTITUTION PARTY,UT
1,2012,H2CA01110,P,"[2012, 2014, 2016]",01,1.0,[01],[2012],False,2012-02-22,...,Challenger,2012-02-22,2012-02-22,2013-04-26T09:04:30,"AANESTAD, SAMUEL",H,House,REP,REPUBLICAN PARTY,CA
2,2018,H8CO06237,C,[2018],06,6.0,[06],[2018],False,2017-04-26,...,Challenger,2017-04-26,2017-04-26,2017-08-01T20:57:28,"AARESTAD, DAVID",H,House,DEM,DEMOCRATIC PARTY,CO
3,2008,P80002926,N,"[2006, 2008, 2010, 2012, 2014, 2016]",00,0.0,[00],[2008],False,2005-10-12,...,Open seat,2007-03-13,2007-03-13,2016-11-17T06:10:48,"AARON, LAURA DAVIS",P,President,DEM,DEMOCRATIC PARTY,US
4,2000,H0MA01024,P,"[2000, 2002, 2004]",01,1.0,[01],[2000],False,2000-02-02,...,Challenger,2000-02-02,2000-02-02,2002-04-12T00:00:00,"ABAIR, PETER JON",H,House,REP,REPUBLICAN PARTY,MA


In [413]:
df_candidate_id = df_general_cand[['name', 
                                   'candidate_id', 
                                   'state',
                                   'incumbent_challenge_full', 
                                   'party_full',
                                   'office_full',
                                   'cycles',
                                   'election_years',
                                   'last_file_date',
                                   'load_date']]
number_candidates = df_candidate_id.shape[0]
print('Number of Candidates Collected: '+  str(number_candidates))
df_candidate_id.head()
#df_candidate_id.name.to_csv('candidate_names.csv')

Number of Candidates Collected: 18615


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,"AANESTAD, SAMUEL",H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,"AARESTAD, DAVID",H8CO06237,CO,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,"AARON, LAURA DAVIS",P80002926,US,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,"ABAIR, PETER JON",H0MA01024,MA,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


In [414]:
#Test
df_candidate_id[df_candidate_id.name == 'AALDERS, TIMOTHY NOEL']


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22


In [585]:
df_candidate_id.to_pickle('candidate_ids.pkl')

In [4]:
df_candidate_id = pd.read_pickle('candidate_ids.pkl')

In [5]:
df_candidate_id

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,"AANESTAD, SAMUEL",H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,"AARESTAD, DAVID",H8CO06237,CO,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,"AARON, LAURA DAVIS",P80002926,US,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,"ABAIR, PETER JON",H0MA01024,MA,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00
5,"ABATE, CAMILLE M",H6NJ05155,NJ,Challenger,DEMOCRATIC PARTY,House,"[2006, 2008]","[2006, 2008]",2007-06-05,2009-04-29T00:00:00
6,"ABATE, FRANK G",H2NJ12036,NJ,Challenger,DEMOCRATIC PARTY,House,"[1992, 1994, 1996, 1998]",[1992],1992-04-15,2002-04-03T00:00:00
7,"ABBAS, JEFFREY LYN",H0IA03071,IA,Challenger,REPUBLICAN PARTY,House,[1990],[1990],1990-05-01,2002-03-30T00:00:00
8,"ABBOTT, BONNIE",H4TX15043,TX,Challenger,REPUBLICAN PARTY,House,[1994],[1994],1994-01-14,2002-03-30T00:00:00
9,"ABBOTT, JAMES W",H6SD00077,SD,Challenger,DEMOCRATIC PARTY,House,"[1996, 1998]",[1996],1995-10-02,2002-04-03T00:00:00


### Construct Committee_id DataFrame B

In [387]:
#Retrieve Comprehensive Committee Dictionary

i = 1
committee_dict = []
while True:
    url = 'https://api.open.fec.gov/v1/committees/\
?per_page=100&sort_null_only=false&sort=name\
&sort_hide_null=false\
&page='+str(i) + '\
&min_first_file_date=2010\
&api_key=' + api_key

    committee_id = requests.get(url)
    if committee_id.json()['results'] ==  []:
        break
    else: 
        results = committee_id.json()['results']
        committee_dict = committee_dict +  results
        i += 1
print('Done')
print(i)

Done
199


In [388]:
df_general_comm = pd.DataFrame(committee_dict)
df_general_comm.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,organization_type,organization_type_full,party,party_full,state,treasurer_name
0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,None,None,W,WRITE-IN,MA,"TEIXEIRA, RASHARNDA SHELINE MISS"
1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",None,None,None,None,DC,ANGELIQUE HARRIS
2,[],C00563023,O,Super PAC (Independent Expenditure-Only),[2014],U,Unauthorized,T,2014-05-12,2014-05-16,2014-06-16,10 ^ 9+,None,None,None,None,CA,RANDI M WILLIS
3,[],C00687368,O,Super PAC (Independent Expenditure-Only),[2018],U,Unauthorized,Q,2018-09-12,2018-09-12,2018-11-02,11/06/18 VICTORY FUND,None,None,None,None,MS,"KEETON, LYNETTE"
4,[],C00686774,H,House,[2018],J,Joint fundraising committee,Q,2018-09-05,2018-09-05,2018-10-21,1106 FUND,None,None,None,None,DC,"MIELNIK, MEGAN"


In [389]:
df_committee_id = df_general_comm.drop(columns=['name', 'organization_type', 'organization_type_full', 'party', 'party_full', 'treasurer_name'])
empty = []
df_committee_id = df_committee_id.loc[df_committee_id.candidate_ids.map(len) > 0].reset_index()
df_committee_id.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-25,DC


In [390]:
# Add column for metric to weight the effect of contribution 
# on each candidate represented by committee

# Simple approach, to be updated at a later point

number_candidates = np.array([len(x) for x in df_committee_id.candidate_ids])**float(-1)
df_committee_id['candidate_donor_weight'] = number_candidates

In [391]:
df_committee_id.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH,0.500
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-25,DC,1.000


In [586]:
df_committee_id.to_pickle('committee_ids.pkl')  

In [7]:
df_committee_id = pd.read_pickle('committee_ids.pkl')

In [8]:
df_committee_id

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,state,candidate_donor_weight
0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,MA,1.000000
1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",DC,1.000000
2,[],C00563023,O,Super PAC (Independent Expenditure-Only),[2014],U,Unauthorized,T,2014-05-12,2014-05-16,2014-06-16,10 ^ 9+,CA,inf
3,[],C00687368,O,Super PAC (Independent Expenditure-Only),[2018],U,Unauthorized,Q,2018-09-12,2018-09-12,2018-11-02,11/06/18 VICTORY FUND,MS,inf
4,[],C00686774,H,House,[2018],J,Joint fundraising committee,Q,2018-09-05,2018-09-05,2018-10-21,1106 FUND,DC,inf
5,[],C00659722,N,PAC - Nonqualified,[2018],J,Joint fundraising committee,T,2017-11-03,2017-11-03,2018-03-08,115TH FRESHMEN VICTORY,VA,inf
6,[],C90013426,I,Independent Expenditor (Person or Group),"[2012, 2014, 2016, 2018]",U,Unauthorized,A,2012-09-25,None,2013-07-15,1199SEIU UNITED HEALTHCARE WORKERS EAST,NY,inf
7,[],C00561084,N,PAC - Nonqualified,[2014],U,Unauthorized,T,2014-04-07,2014-04-07,2014-08-05,11 CONGRESSIONAL DISTRICT OF ILLINOIS TEA PART...,IL,inf
8,[],C00538975,Y,Party - Qualified,"[2012, 2014, 2016, 2018]",U,Unauthorized,Q,2012-12-03,2016-09-02,2018-10-24,11TH CONGRESSIONAL DISTRICT OF VIRGINIA REPUBL...,VA,inf
9,[],C00542720,Y,Party - Qualified,"[2014, 2016, 2018]",U,Unauthorized,Q,2013-03-14,2015-02-23,2018-10-18,12TH CONGRESSIONAL DISTRICT DEMOCRATS,MI,inf


## Retrieve Committee Donors 

### Takes too long, extraneous information, inefficient approach

In [ ]:
#    "per_page": 100,
#    "pages": 507467,
#    "count": 50746611

i = 0
last_index = 0
donor_dict = []
while True:
    url = 'https://api.open.fec.gov/v1/schedules/schedule_a/?\
per_page=100\
&last_index={}\
&sort_null_only=false\
&is_individual=true\
&min_date=2010\
&two_year_transaction_period=2018\
&sort_hide_null=false\
&api_key={}'.format(last_index, api_key) 
    
    donor = requests.get(url)
    
    
    try:
        print(donor.json())
    except json.decoder.JSONDecodeError:
        print("Encountered JSON Decoder Error")
        break
    
    if (donor.status_code == 429):
        print('429 Response Received. Sleep for an hour')
        print(datetime.datetime.now())
        print(i)
        print(last_index)
        time.sleep(3610) #Sleep for an hour & 10 Seconds before continuing.
        print('Starting up again')
        candidate_committee = requests.get(url)
        if candidate_committee.status_code == 429:
            break;
    
    print(last_index)
    print(i)
    donor = donor.json()
    donor_results = donor['results']
    
    if donor_results == []:
        break
    else: 
        donor_dict = donor_dict + donor_results 
        last_index_contribution_date = [donor['pagination']['last_indexes']['last_index'],
                                       donor['pagination']['last_indexes']['last_contribution_receipt_date']]
        last_index = last_index_contribution_date[0]
        i += 1
        
    
donor_dict

In [83]:
donor_df = pd.DataFrame(donor_dict)
columns = ['contribution_receipt_amount', 
      'contribution_receipt_date',
      'receipt_type_desc',
      'entity_type_desc',
      'contributor_city',
      'contributor_state',
      'contributor_name',
      'party_full',
      'candidate_id',
      'committee_id',
      'committee',
      'line_number_label',
      'fec_election_type_desc']
donor_df = donor_df[columns]


## Better Approach

In [167]:
# Use by_state/by_candidate, by_size/by_candidate
# QUESTION, Does this by_candidate search include donations made to committee's? 

# Use /schedules/schedule_a/by_size/by_candidate/
# Use /schedules/schedule_a/by_state/by_candidate/


# by_size/by_candidate
#  -0    $200 and under
#  -200  $200.01 - $499.99
#  -500  $500 - $999.99
#  -1000 $1000 - $1999.99
#  -2000 $2000 +

def cand_donor_size(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings

    url_size = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_size/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)

    donor_cand_size = requests.get(url_size)
    #print(donor_cand_size.status_code)
    donor_size_result = donor_cand_size.json()['results']
    df_dcand_size = pd.DataFrame(donor_size_result)
    return df_dcand_size

    
    
# Schedule A individual receipts aggregated by contributor state.
# This is an aggregate of only individual contributions. 
def cand_donor_state(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings
    url_state = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_state/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)
    
    donor_cand_state = requests.get(url_state)
    #print(donor_cand_state.status_code)
    donor_state_result = donor_cand_state.json()['results']
    df_dcand_state = pd.DataFrame(donor_state_result)
    return df_dcand_state



In [168]:
#Test
a = cand_donor_state('P80001571', 2016)
cand_donor_state('P80001571', 2016)


,candidate_id,cycle,state,state_full,total
0,P80001571,2016,AA,Armed Forces Americas,476.75
1,P80001571,2016,AE,Armed Forces Europe,7579.01
2,P80001571,2016,AK,Alaska,368816.42
3,P80001571,2016,AL,Alabama,1674759.75
4,P80001571,2016,AP,Armed Forces Pacific,2435.77
5,P80001571,2016,AR,Arkansas,739125.42
6,P80001571,2016,AS,American Samoa,4684.00
7,P80001571,2016,AZ,Arizona,2771362.70
8,P80001571,2016,CA,California,12082510.57
9,P80001571,2016,CO,Colorado,2275468.86


In [169]:
# Presidential Election, Likeley has donations from all entities within the United States. 
states = a.state_full
#states = np.append(states, 'Armed Forces Americas')
states.head()

0    Armed Forces Americas
1      Armed Forces Europe
2                   Alaska
3                  Alabama
4     Armed Forces Pacific
Name: state_full, dtype: object

In [170]:
cand_donor_size('P80001571', 2018)


,candidate_id,cycle,size,total
0,P80001571,2018,0,19373283.84
1,P80001571,2018,200,3405918.70
2,P80001571,2018,500,1179909.12
3,P80001571,2018,1000,1141544.03
4,P80001571,2018,2000,5518142.30


In [171]:
# Use /committee/{committee_id}/schedules/schedule_a/by_size/
# Use /committee/{committee_id}/schedules/schedule_a/by_state/
# Extract candidate_ids supported by each committee, use DataFrame B

def comm_donor_size(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_size/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_size = requests.get(url)
    #print(donor_comm_size.status_code)
    donor_comm_size = donor_comm_size.json()['results']
    df_dcomm_size = pd.DataFrame(donor_comm_size)
    return df_dcomm_size

def comm_donor_state(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_state/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_state = requests.get(url)
    #print(donor_comm_state.status_code)
    donor_comm_state = donor_comm_state.json()['results']
    df_dcomm_state = pd.DataFrame(donor_comm_state)
    return df_dcomm_state



In [515]:
#Test 
comm_donor_state('C00624759')

KeyError: 'results'

In [173]:
comm_donor_size('C00624759')

,committee_id,count,cycle,size,total
0,C00624759,21.0,2016,200,5300.0
1,C00624759,NaN,2016,0,675.0
2,C00624759,24.0,2016,1000,25500.0
3,C00624759,5.0,2016,500,2500.0
4,C00624759,12.0,2016,2000,44200.0


## Compile Comprehensive DataFrame of Donors per Candidate

### 1. For Donations made directly to a candidate


### 2. For Donations made to a committee that has a list of candidates that they support.

In [242]:
df_candidate_id.head()

,name,candidate_id,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,"AANESTAD, SAMUEL",H2CA01110,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,"AARESTAD, DAVID",H8CO06237,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,"AARON, LAURA DAVIS",P80002926,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,"ABAIR, PETER JON",H0MA01024,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


In [175]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [574]:
# Input: Candidate Name, Election Type, Date-time of election (Use Pandas Date time Format)
# 1. Will incorporate a funcation call to 
# 2. to retrieve the fraction of committee donations for that candidate

# by_size/by_candidate
#  -0    $200 and under Level 1 
#  -200  $200.01 - $499.99 Level 2 
#  -500  $500 - $999.99 Level 3 
#  -1000 $1000 - $1999.99 Level 4
#  -2000 $2000 + Level 5

a = cand_donor_state('P80001571', 2016)
states = a.state_full


# Returns rows, 1 row per cycle with all features for one candidate. 
def candidate_donor_df(candidate_name, states, df_candidate_id):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    columns = np.append(cand_info, states)
    columns = np.append(columns, donation_levels)

    df = pd.DataFrame(columns=columns)
    
    for cand_id in cand_ids:
        #print(cand_id)
    
        cycles = cand_row.cycles.values[0]


        

        #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
        
        #print(cycles)
        for cycle in cycles:
            df_size = cand_donor_size(cand_id, cycle)
            df_state = cand_donor_state(cand_id, cycle)
            if df_size.values.size == 0 or df_state.values.size == 0:
                continue

            donor_states = df_state.state_full

            #Build row of information (donor_states)

            # Idea
            # Create dictionary with states as keys and initialized with 0s as values. 
            initialize_array = np.zeros(len(states))
            state_dict = dict(zip(states, initialize_array))

            for ds in donor_states:            
                # Overwrite select values with information from df_state
                state_dict[ds] = df_state[df_state.state_full == ds].total.values[0]


            # Add size features to state_dict 


            level_array = np.zeros(5)
            level_dict = dict(zip(donation_levels, level_array))

            for level in df_size['size']:
                if level == 0:
                    donation_level = 'Donation Level 1'
                if level == 200:
                    donation_level = 'Donation Level 2'
                if level == 500:
                    donation_level = 'Donation Level 3'
                if level == 1000:
                    donation_level = 'Donation Level 4'
                if level == 2000:
                    donation_level = 'Donation Level 5'
                level_dict[donation_level] = df_size[df_size['size'] == level].total.values[0]


            cand_info_dict = {'name': candidate_name, 
                              'cand_id': cand_id, 
                              'state': cand_row.state.values[0],
                              'cycle': cycle,
                              'incumbent': cand_row.incumbent_challenge_full.values[0],
                              'office_full': cand_row.office_full.values[0],
                              'party': cand_row.party_full.values[0],
                              'committee_id': 0}
            state_dict = merge_two_dicts(cand_info_dict, state_dict)
            state_dict = merge_two_dicts(state_dict, level_dict)
            #return state_dict


            #return state_dict.values()
            df.loc[j] = list(state_dict.values()) # An array of the row information
            j += 1
    return df

                         
                         
        

In [575]:
#b = candidate_donor_df('PELOSI, NANCY', states)
b = candidate_donor_df('TRUMP, DONALD J', states, df_candidate_id)
b

,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,Armed Forces Americas,Armed Forces Europe,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,"TRUMP, DONALD J",P80001571,US,2016,Incumbent,President,REPUBLICAN PARTY,0,476.75,7579.01,...,2193577.67,1282663.70,465286.53,387069.81,67034.39,1.123466e+08,27087044.46,15583477.52,13192364.05,27020664.65
1,"TRUMP, DONALD J",P80001571,US,2018,Incumbent,President,REPUBLICAN PARTY,0,0.00,975.00,...,282388.46,433287.63,36927.37,41374.93,11381.05,1.937328e+07,3405918.70,1179909.12,1141544.03,5518142.30


# Committe's Have additional donations which are not represented by the Candidate Donations. 

In [393]:
len(df_committee_id.committee_id)
#df_committee_id

8850

In [398]:
# Run this, build committee donor extraction algorithm that adds to the other function. 
# Run for all candidates. Try and find the election results first tho.

# Ran in a separate notebook
def reformat_committee_df(df):
    df_temp = df.copy()
    df_comm = pd.DataFrame(columns = df_temp.columns)
    i = 1
    for list_cand_id in df_temp.candidate_ids:
        print(i)
        for cand_id in list_cand_id:
            df_temp.iloc[i-1:i,:] = df_temp.iloc[i-1:i,:].drop(columns='candidate_ids')
            df_temp.iloc[i-1:i,:]['candidate_ids'] = cand_id
            df_comm = pd.concat([df_comm, df_temp.iloc[i-1:i,:]])
        i+=1
        if i == 100:
            break
    return df_comm



In [492]:
#df_comm = reformat_committee_df(df_committee_id)


In [493]:
df_comm = pd.read_pickle('candidate_id_formatted_v1.pkl')
df_comm.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,P00008052,C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000
1,1,S8MN00438,C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000
2,30,S0MA00109,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
2,30,S2NV00183,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
2,30,S4IN00014,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125


In [494]:
cand_row = df_candidate_id[df_candidate_id.candidate_id == 'S8MS00196']
cand_name = cand_row.name.values[0]
cand_name


'WICKER, ROGER F'

In [487]:
cand_row = df_candidate_id[df_candidate_id.name == cand_name]
cand_id = cand_row.candidate_id.values
cand_row

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
17936,"WICKER, ROGER F",S8MS00196,MS,Incumbent,REPUBLICAN PARTY,Senate,"[2008, 2010, 2012, 2014, 2016, 2018]","[2008, 2012, 2018]",2013-10-14,2018-02-17T09:16:20


In [576]:
#Add df_candidate_id input to the previous function for candidate donor information collection function 

def get_committee_candidates(candidate_name, states, df_candidate_id, df_comm):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    columns = np.append(cand_info, states)
    columns = np.append(columns, donation_levels)

    df = pd.DataFrame(columns=columns)
    #print(cand_ids)
    for cand_id in cand_ids:
        comm_rows = df_comm[df_comm.candidate_ids == cand_id]
        #return comm_rows
        
        #for row in comm_rows: 
        for index, row in comm_rows.iterrows(): #Iterate through each row of the dataframe comm_row
            #print('iter')
            candidate_donor_weight = row.candidate_donor_weight
            cycles = row.cycles
            comm_id = row.committee_id
            #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
            #print(cycles)
            for cycle in cycles:
                df_size = comm_donor_size(comm_id)
                df_state = comm_donor_state(comm_id)
                if df_size.values.size == 0 or df_state.values.size == 0:
                    continue

                donor_states = df_state.state_full

                #Build row of information (donor_states)

                # Idea
                # Create dictionary with states as keys and initialized with 0s as values. 
                initialize_array = np.zeros(len(states))
                state_dict = dict(zip(states, initialize_array))

                for ds in donor_states:            
                    # Overwrite select values with information from df_state
                    state_row = df_state[df_state.state_full == ds]
                    state_dict[ds] = state_row.total.values[0] * candidate_donor_weight


                # Add size features to state_dict 


                level_array = np.zeros(5)
                level_dict = dict(zip(donation_levels, level_array))

                for level in df_size['size']:
                    if level == 0:
                        donation_level = 'Donation Level 1'
                    if level == 200:
                        donation_level = 'Donation Level 2'
                    if level == 500:
                        donation_level = 'Donation Level 3'
                    if level == 1000:
                        donation_level = 'Donation Level 4'
                    if level == 2000:
                        donation_level = 'Donation Level 5'
                    size_row = df_size[df_size['size'] == level]
                    level_dict[donation_level] = size_row.total.values[0] * candidate_donor_weight


                cand_info_dict = {'name': candidate_name, 
                                  'cand_id': cand_id, 
                                  'state': row.state,
                                  'cycle': cycle,
                                  'incumbent': 0,
                                  'office_full': 0,
                                  'party': 0,
                                  'committee_id': row.committee_id}
                state_dict = merge_two_dicts(cand_info_dict, state_dict)
                state_dict = merge_two_dicts(state_dict, level_dict)

                #return state_dict


                #return state_dict.values()
                df.loc[j] = list(state_dict.values()) # An array of the row information
                j += 1
    return df
    


In [512]:
get_committee_candidates('TRUMP, DONALD J', states, df_candidate_id, df_comm)
#get_committee_candidates('MAFFEI, DANIEL BENJAMIN MR.', states, df_candidate_id, df_comm)

#get_committee_candidates('WICKER, ROGER F', states, df_candidate_id, df_comm)

,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,Armed Forces Americas,Armed Forces Europe,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,"TRUMP, DONALD J",P80001571,NY,2016,0,0,0,C00580100,0.0,0.0,...,2193577.67,1282663.7,465286.53,387069.81,67034.39,19373283.84,3405918.7,15583477.52,13192364.05,5518142.3
1,"TRUMP, DONALD J",P80001571,NY,2018,0,0,0,C00580100,0.0,0.0,...,2193577.67,1282663.7,465286.53,387069.81,67034.39,19373283.84,3405918.7,15583477.52,13192364.05,5518142.3


# Get Donor Information for Every Candidate From DF A & B

In [544]:
len(df_comm)

10420

In [577]:
cand_info = ['name', 'cand_id', 'state','cycle', 'incumbent', 'office_full', 'party', 'committee_id']
donation_levels = ['Donation Level 1',
       'Donation Level 2', 
       'Donation Level 3',
       'Donation Level 4',
       'Donation Level 5']
columns = np.append(cand_info, states)
columns = np.append(columns, donation_levels)
df = pd.DataFrame(columns=columns)
i = 1
amount = len(df_candidate_id.name) 
for name in df_candidate_id.name:
    df_name = candidate_donor_df(name, states, df_candidate_id)
    df = pd.concat([df, df_name], sort=False)
    
    df_name_comm = get_committee_candidates(name, states, df_candidate_id, df_comm)
    df = pd.concat([df, df_name_comm], sort=False)

    print('{}/{}'.format(i, amount))
    i+=1
    if i == 10:
        break
df

1/18615
2/18615
3/18615
4/18615
5/18615
6/18615
7/18615
8/18615
9/18615


,name,cand_id,state,cycle,incumbent,office_full,party,committee_id,Armed Forces Americas,Armed Forces Europe,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2012,Open seat,Senate,CONSTITUTION PARTY,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
1,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2018,Open seat,Senate,CONSTITUTION PARTY,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4372.18,1000.00,0.0,1723.23,2400.0
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2012,0,0,0,C00512632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
1,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2014,0,0,0,C00512632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
2,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2016,0,0,0,C00512632,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0
3,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,2018,0,0,0,C00678300,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4372.18,1000.00,0.0,1723.23,2400.0
0,"AANESTAD, SAMUEL",H2CA01110,CA,2012,Challenger,House,REPUBLICAN PARTY,0,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0
0,"AANESTAD, SAMUEL",H2CA01110,CA,2012,0,0,0,C00513580,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0
1,"AANESTAD, SAMUEL",H2CA01110,CA,2014,0,0,0,C00513580,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0
2,"AANESTAD, SAMUEL",H2CA01110,CA,2016,0,0,0,C00513580,0.0,0.0,...,0.0,800.0,0.0,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0


In [582]:
cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
sum_columns = np.append(states, donation_levels)
sum_columns

col = ['name','cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party', 'committee_id']
df_grouped = df.groupby(cand_info)[sum_columns].sum()

In [583]:
df_grouped.iloc[:,:]

Armed Forces Americas  \
name                  cand_id   state cycle incumbent  office_full party              committee_id                          
AALDERS, TIMOTHY NOEL S2UT00229 UT    2012  0          0           0                  C00512632                       0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                               0.0   
                                      2014  0          0           0                  C00512632                       0.0   
                                      2016  0          0           0                  C00512632                       0.0   
                                      2018  0          0           0                  C00678300                       0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                               0.0   
AANESTAD, SAMUEL      H2CA01110 CA    2012  0          0           0                  C00513580                       0.0   
                                            Challenger House       REPUBLICAN PARTY   0                               0.0   
                                      2014  0          0           0                  C00513580                       0.0   
                                      2016  0          0           0                  C00513580                       0.0   
AARESTAD, DAVID       H8CO06237 CO    2018  0          0           0                  C00637678                       0.0   
                                            Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABAIR, PETER JON      H0MA01024 MA    2000  Challenger House       REPUBLICAN PARTY   0                               0.0   
                                      2002  Challenger House       REPUBLICAN PARTY   0                               0.0   
ABATE, CAMILLE M      H6NJ05155 NJ    2006  Challenger House       DEMOCRATIC PARTY   0                               0.0   
                                      2008  Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABATE, FRANK G        H2NJ12036 NJ    1992  Challenger House       DEMOCRATIC PARTY   0                               0.0   
ABBAS, JEFFREY LYN    H0IA03071 IA    1990  Challenger House       REPUBLICAN PARTY   0                               0.0   

                                                                                                    Armed Forces Europe  \
name                  cand_id   state cycle incumbent  office_full party              committee_id                        
AALDERS, TIMOTHY NOEL S2UT00229 UT    2012  0          0           0                  C00512632                     0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                             0.0   
                                      2014  0          0           0                  C00512632                     0.0   
                                      2016  0          0           0                  C00512632                     0.0   
                                      2018  0          0           0                  C00678300                     0.0   
                                            Open seat  Senate      CONSTITUTION PARTY 0                             0.0   
AANESTAD, SAMUEL      H2CA01110 CA    2012  0          0           0                  C00513580                     0.0   
                                            Challenger House       REPUBLICAN PARTY   0                             0.0   
                                      2014  0          0           0                  C00513580                     0.0   
                                      2016  0          0           0                  C00513580                     0.0   
AARESTAD, DAVID       H8CO06237 CO    2018  0          0           0                  C00637678                     0.0   


## Retrive Basic Candidate Information by Candidate ID

In [ ]:
#Retrieve Basic Candidate Information
def retrieve_candidate_information(candidate_id):
    url = 'https://api.open.fec.gov/v1/candidate/' + str(candidate_id) + '/?sort=name&per_page=20&page=1&api_key=' + api_key
    temp = requests.get(url)
    results = temp.json()['results']
    return pd.DataFrame(results)

#Retrieve Total/Financial Candidate Information
def retrieve_candidate_totals(candidate_id):
    url = 'https://api.open.fec.gov/v1/candidate/' + str(candidate_id) + '/totals/?api_key='+ api_key +'&sort=-cycle&per_page=20&page=1'
    temp = requests.get(url)
    results = temp.json()['results']
    return pd.DataFrame(results)
    

## Find Nancy Pelosi Data 
Full name: Nancy Patricia D'Alesandro Pelosi

In [77]:
df_candidate_id.loc[df_candidate_id['name'].str.contains('PELOSI')]

,name,candidate_id,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
12944,"PELOSI, NANCY",H8CA05035,Incumbent,DEMOCRATIC PARTY,House,"[1988, 1990, 1992, 1994, 1996, 1998, 2000, 200...","[1988, 1990, 1992, 1994, 1996, 1998, 2000, 200...",2016-11-16,2018-04-05T21:08:42


In [ ]:
retrieve_candidate_information('H8CA05035')

In [ ]:
retrieve_candidate_totals('H8CA05035')

## Retrieve Contributions by donor

### Construct Comittee with Candidate DataFrame